In [14]:
R.version

               _                           
platform       x86_64-apple-darwin17.0     
arch           x86_64                      
os             darwin17.0                  
system         x86_64, darwin17.0          
status                                     
major          4                           
minor          0.0                         
year           2020                        
month          04                          
day            24                          
svn rev        78286                       
language       R                           
version.string R version 4.0.0 (2020-04-24)
nickname       Arbor Day                   

## The Expander test
The following is based on the pre-print by Stefan Steinerberger *Using expander graphs to test whether samples are I.I.D.* ( https://arxiv.org/pdf/2008.01153.pdf )
Given a sequence $x_1, \dots, x_n$, a specific graph $G$ associated to this sequence is generated. The adjacency matrix of this graph has eigen values $\lambda_1 > \lambda_2 > \dots > \lambda_n$. We compute $\lambda = \max\lbrace \lambda_2, \lambda_n\rbrace$. The larger $\lambda - 2 \sqrt{3}$ is, the less likely the sequence is to be random

In [1]:
require(igraph)

Loading required package: igraph

Warning message:
“package ‘igraph’ was built under R version 4.0.2”

Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union




Here we construct the 4-regular graph associated to our data (read abstract or introduction of the pre-print to see how this is built). 

In [2]:
construct_graph <- function(x){
    n <- length(x)
    g <- make_empty_graph(n = n)
    
    g <- add_edges(g, c(n, 1))
    for (i in 1:(n-1)){
        g <- add_edges(g, c(i, i+1))
    }
    
    g <- add_edges(g, c(order(x)[n], order(x)[1]))
    for (i in 1:(n-1)){
        g <- add_edges(g, c(order(x)[i], order(x)[i+1]))
    }
    
    g <- as.undirected(g, mode="each")
    
    return(g)
}

Next we calculate the desired $\lambda$

In [3]:
lambda <- function(g){
    evalues <- eigen(as_adjacency_matrix(g), only.values=TRUE)$values
    evalues <- sort(evalues, decreasing = TRUE)
    a <- abs(evalues[2])
    b <- abs(tail(evalues, 1))
    lambda <- max(a, b)
    return(lambda)
}

We wrap it up in a function which tell the difference between our $\lambda$ and $2 \sqrt{3}$. 

In [11]:
deviation <- function(x){
    g <- construct_graph(x)
    l <- lambda(g)
    return(abs(l - 2*sqrt(3)))
}

We test it with "bad" data: a congruential generator by Lehmer, and numbers coming from distinct distributions.

In [15]:
value <- 47594118

lehmer <- c(value)

for(i in 1:499){
    value <- (23*value) %%( 10**8 + 1)
    lehmer <- c(lehmer, value)
}

In [16]:
deviation(lehmer)

[1] 0.08405181

In [23]:
x <- c(rnorm(100), runif(100), rpois(100,lambda = 1), rgeom(100, prob = 1), rexp(100))

In [24]:
deviation(x)

[1] 0.5340322

We test it with numbers coming from the same distribution:

In [25]:
deviation(rnorm(1000))

[1] 0.001009767

In [26]:
deviation(runif(1000))

[1] 0.001793863

In [ ]:
rpois(10, lam)